In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim


model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True).to(device)

In [2]:
df = pd.read_csv('../Final_Resume_Texts/resume_texts.csv')

In [ ]:
roles = {}
id_role = {}

for i, row in df.iterrows():
    role = row['title']
    id = row['id']

    if role in roles:
        id_role[id] = role
    role_key = None
    max_sim = -1
    for key in roles:
        sentences = [role, key]
        embeddings = model.encode(sentences, convert_to_tensor=True)
        cosine_scores = cos_sim(embeddings[0], embeddings[1])
        if cosine_scores > max_sim:
            max_sim = cosine_scores
            role_key = key
    if max_sim >= 0.8:
        id_role[id] = role_key
        roles[role_key].add(role)
    else:
        roles[role] = set()
        id_role[id] = role


In [ ]:
# for each id in id_role, get the role and add it to the dataframe
df['role'] = df['id'].apply(lambda x: id_role[x])

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
#
analysis_df = pd.read_csv('../Final_Resume_Texts/resume_with_raw_roles_0_500.csv')
roles = {}
for i, row in analysis_df.iterrows():
    role = row['role_raw']
    if role in roles:
        roles[role].add(row['title'])
    else:
        roles[role] = set()
        roles[role].add(row['title'])

new_roles = {}
for role in roles:
    new_roles[role] = list(roles[role])

In [12]:
import json
with open('roles.json', 'w') as f:
    json.dump(new_roles, f)